In [7]:
import pandas as pd
import joblib
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

# Load the pre-trained model
mlp = joblib.load('model1 - Copy.pkl')

# Load training data
df_train = pd.read_csv("train.csv")

# Drop irrelevant columns
to_remove = ["ID","SEQUENCE","FUNCTION","SYNONYMS","UNIPROID","PUBCHCID","Accession Number"]
X1 = df_train.drop(columns=to_remove).drop(columns=["Target_Status"])

# Label encode the target
le = LabelEncoder()
Y1 = le.fit_transform(df_train["Target_Status"])

# Limit one-hot encoding to columns with fewer unique values, apply label encoding to high cardinality columns
high_cardinality_columns = ['TargetID']  # Add other columns if needed
for col in high_cardinality_columns:
    X1[col] = le.fit_transform(X1[col])

# One-hot encode the rest of the columns with low cardinality
X1 = pd.get_dummies(X1, drop_first=True)

# Apply PCA to reduce dimensionality
pca = PCA(n_components=100)  # Reduce dimensions to 100 principal components
X1_reduced = pca.fit_transform(X1)

# Sample a portion of the data to reduce load
X1_sampled, _, Y1_sampled, _ = train_test_split(X1_reduced, Y1, test_size=0.7, random_state=42)

# Define a smaller hyperparameter grid for GridSearchCV
param_grid = {
    'hidden_layer_sizes': [(50,), (100,)],  # Reduce options
    'activation': ['relu'],  # Focus on one activation function
    'solver': ['adam'],  # Use a single solver
    'alpha': [0.0001, 0.001],  # Regularization strength
    'learning_rate': ['constant']  # One learning rate schedule
}

# Perform grid search with reduced cross-validation folds
grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

# Fit the model on the reduced dataset
grid_search.fit(X1_sampled, Y1_sampled)

# Print the best hyperparameters
print(f"Best hyperparameters: {grid_search.best_params_}")

# Get the best model from grid search
best_mlp = grid_search.best_estimator_

# Save the optimized model
joblib.dump(best_mlp, 'optimized_model.pkl')


KeyboardInterrupt: 

In [6]:
import pandas as pd


X1
# Load the test dataset
df_test = pd.read_csv("test.csv")

# Drop the 'ID' column and create dummy variables for categorical features
df_test_n = pd.get_dummies(df_test.drop(columns=["ID"]))

# Identify missing columns that are in the training set but not in the test set
missing_cols = set(X1.columns) - set(df_test_n.columns)

# Add missing columns to the test DataFrame and set their values to 0
for col in missing_cols:
    df_test_n[col] = 0

# Reorder columns to match the training set
df_test_n = df_test_n[X1.columns]

# Now df_test_n is ready for prediction using your model




NameError: name 'X1' is not defined

In [9]:
import pandas as pd
import joblib

# Load the optimized model
best_mlp = joblib.load('optimized_model.pkl')

# Load the test dataset
df_test = pd.read_csv("test.csv")

# Drop the 'ID' column and create dummy variables
df_test_n = pd.get_dummies(df_test.drop(columns=["ID"]))

# Identify missing columns that are in the training set but not in the test set
missing_cols = set(X1.columns) - set(df_test_n.columns)

# Add missing columns to the test DataFrame and set their values to 0
for col in missing_cols:
    df_test_n[col] = 0

# Reorder columns to match the training set
df_test_n = df_test_n[X1.columns]

# Debugging: Print the shapes of X1 and df_test_n
print(f"Training set shape: {X1.shape}")
print(f"Test set shape: {df_test_n.shape}")

# Make predictions using the optimized model
try:
    predictions = best_mlp.predict(df_test_n)
except ValueError as e:
    print("Error during prediction:", e)
    raise

# If you want to save the predictions along with the original IDs
df_results = pd.DataFrame({
    "ID": df_test["ID"],  # Include the ID for reference
    "Predicted_Status": predictions
})

# Save the results to a CSV file
df_results.to_csv("predictions.csv", index=False)

print("Predictions have been saved to 'predictions.csv'.")


C:\Users\jaind\AppData\Local\Temp\ipykernel_14972\1345348991.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test_n[col] = 0
C:\Users\jaind\AppData\Local\Temp\ipykernel_14972\1345348991.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test_n[col] = 0
C:\Users\jaind\AppData\Local\Temp\ipykernel_14972\1345348991.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(

Training set shape: (134486, 7705)
Test set shape: (57637, 7705)


C:\Users\jaind\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(


Error during prediction: X has 7705 features, but MLPClassifier is expecting 100 features as input.


ValueError: X has 7705 features, but MLPClassifier is expecting 100 features as input.